In [1]:
import numpy as np 
import pygame
import time

pygame 2.4.0 (SDL 2.26.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Circle:
    
   
    def __init__(self,x, y, mass, distance, color,radius, angle):
        self.x = x
        self.y = y
        self.mass = mass
        self.distance = distance
        self.color = color
        self.radius = radius
        self.angle = angle
          
        
    def change_angle(self, angle):    
        self.angle += angle
        if self.angle >= 2 * np.pi:
            self.angle -= 2 * np.pi
            
            
    def set_coords(self, refference_point,to='platform_center_point'):
      
        distance = get_distance(refference_point, [self.x,self.y]) if to=='barycenter_point' else self.distance 
        self.x = refference_point[0] + distance * np.cos(self.angle)
        self.y = refference_point[1] - distance *np.sin(self.angle)
        
        
    def change_distance(self, unit_displace):
        
        if 0<= self.angle < np.pi:
            self.distance -= unit_displace
        else:
            self.distance += unit_displace

In [3]:
def get_distance(point1, point2):
    return np.sqrt((point2[0] - point1[0]) ** 2 + (point2[1] - point1[1]) ** 2)

In [4]:
def calc_barycenter(bodies, platform_point):
    
    all_bodies = bodies.copy()
    all_bodies.append(platform_point)
    masses = []
    x = []
    y = []
    for i in range(len(all_bodies)):
        body = all_bodies[i]
        x.append(body.x)
        y.append(body.y)
        masses.append(body.mass)
    masses = np.array(masses)
    masses_sum = np.sum(masses)
    bodies_x = np.array(x)
    bodies_y = np.array(y)
    
    bary_x = np.dot(bodies_x, masses)/masses_sum
    bary_y = np.dot(bodies_y, masses)/masses_sum
    return bary_x, bary_y

In [5]:
def create_moving_circles(screen_center, Circle, count, 
                          unit_angle, mass, distance, 
                          color, rotating_circles_radius,center_point_radius):
    unit_distance = distance / (count/2)
    circles = []
    opposite_circles = []
    for i in range(int(count/2)):
        angle = i * unit_angle
        opposite_angle = angle + np.pi
       
        curr_circle_distance = distance - i * unit_distance
        distance_to_opp_point = curr_circle_distance - distance + rotating_circles_radius + center_point_radius
      
        
        circle_coords = (screen_center[0] + curr_circle_distance* np.cos(angle), 
                          screen_center[1] - curr_circle_distance*np.sin(angle))
        
        opposite_coords = (screen_center[0] + distance_to_opp_point* np.cos(opposite_angle), 
                          screen_center[1] - distance_to_opp_point*np.sin(opposite_angle))
        #def __init__(self,x, y, mass, distance, color,radius, angle, decrease = True):
        
        circles.append(Circle(*circle_coords, mass,curr_circle_distance, 
                              color, rotating_circles_radius, angle,))
        
        opposite_circles.append(Circle(*opposite_coords, mass, abs(distance_to_opp_point), 
                                       color,rotating_circles_radius, opposite_angle))
    return circles + opposite_circles

In [6]:
def update_circles_coords(circles_list, unit_displace, delta_angle,refference_point,to='platform_center_point'):
    for circle in circles_list:
        # circle.dist_to_rp = get_distance(refference_point, [circle.x, circle.y])
        circle.change_angle(delta_angle)
        circle.change_distance(unit_displace)
        
        circle.set_coords(refference_point, to=to)

In [7]:
def calc_displacements(circle_mass, platform_mass, unit_displace):
    min_displacement = unit_displace/ (circle_mass + platform_mass)
    
    circle_displacement = min_displacement * platform_mass
    platform_displacement = min_displacement * circle_mass
    
    return circle_displacement, platform_displacement

In [8]:
def get_angle_between_points(point1, point2):
    
    tangens = (point2[1]-point1[1])/(point2[0] - point1[0])
    
    angle = np.arctan(tangens)
    
    final_angle = np.pi + angle
    
    return final_angle

In [9]:
def main_anim(screen_width=1500, screen_height=800):
    screen_center = (screen_width / 2, screen_height / 2)
    FPS = 50
    
    pygame.init()
    clock = pygame.time.Clock()
    screen = pygame.display.set_mode((screen_width, screen_height))
  
    platform_mass = 12
    platform_color = (0, 0, 0)
    platform_point_radius = 6
    
    # def __init__(self,x, y, mass, distance, color,radius, angle)
    
    platform_point = Circle(screen_center[0], screen_center[1], 
                            platform_mass, 0, platform_color, platform_point_radius,
                            0)

    
    rotating_circles_count = 4
    unit_angle = 2 * np.pi / rotating_circles_count
    delta_angle = 2 * np.pi / (FPS * 4)
    
    circle_mass = 1
    color = (0, 0, 255)
    rotating_circles_radius = 4
    center_point_radius = 4
    distance = 500 - center_point_radius - rotating_circles_radius # In pixels
    unit_displace = (distance-10)/ (FPS * 2)
    
    circle_displacement, platform_displacement = calc_displacements(circle_mass, platform_mass,unit_displace)
    
    
    circles_list = create_moving_circles(screen_center, Circle,
                                         rotating_circles_count,
                                         unit_angle, circle_mass, distance,
                                         color, rotating_circles_radius,
                                         center_point_radius)
    
    barycenter_point = calc_barycenter(circles_list, platform_point)
    
    # Reinitialization of the angle and the distance between platform point and barycenter point
    platform_point.angle = get_angle_between_points((platform_point.x, platform_point.y), barycenter_point)
    platform_point.distance = get_distance((platform_point.x, platform_point.y), barycenter_point)
    
    
    
    # Draw circles
    for curr_circle in circles_list:
        pygame.draw.circle(screen, (255, 0, 255), (curr_circle.x, curr_circle.y), curr_circle.radius)
    
 
     
    
    running = True
    
 
    
    while running:
        
        clock.tick(FPS)
        screen.fill((255, 255, 255))  # Clear the screen for each frame
      
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
          
        
        # update_circles_coords(circles_list, unit_displace, delta_angle,[platform_point.x, platform_point.y])
        
        # Rotate the circles over barycenter point with no change of the distances to it
        update_circles_coords(circles_list + [platform_point], 0, delta_angle,barycenter_point,to='barycenter_point')
        
        
         
        
        # barycenter_point = calc_barycenter(circles_list, platform_point)
        
        # Draw barycenter
        pygame.draw.circle(screen, platform_point.color,
                           (barycenter_point[0], barycenter_point[1]),
                           platform_point.radius)
      
       
        #Draw center point
        pygame.draw.circle(screen, (255, 0, 0), (screen_center[0], screen_center[1]), center_point_radius)
        
        # Draw circles
        for curr_circle in circles_list:
            pygame.draw.circle(screen, (255, 0, 255), (curr_circle.x, curr_circle.y), curr_circle.radius)
           
            
        pygame.display.update()
        # time.sleep(0.5)
    
    pygame.quit()

In [10]:
main_anim()

In [11]:
a, b = calc_displacements(1, 3, 10)

print(a, b)


7.5 2.5
